<a href="https://colab.research.google.com/github/ozturkcemal/SupplyChainAnalytics/blob/main/04_SetCovering_Packing_Partitioning/01_SetCovering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing library to read xlsx files
!pip install pandas openpyxl

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

# Read the data from the Excel file, specifying the correct header row
df = pd.read_excel('input.xlsx', header=1)  # This assumes the second row (index 1) is your header

# Drop the 'Sites' column, as requested
df = df.loc[:, df.columns != 'Sites']

# Print the cleaned dataframe
print(df.to_string())

In [55]:
# Create a list 'sites' with numbers from 0 to the number of rows in the dataframe
sites = list(range(0, len(df)))
# Print the list 'sites' to verify
print("sites",sites)
# Create a list 'districts' with numbers from 0 to the number of columns in the dataframe
districts = list(range(0, len(df.columns)))
# Print the list 'districts' to verify
print("districts:",districts)
# Export the dataframe to a CSV file and open it in an editor
#df.to_csv('output.csv')

sites [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
districts: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [ ]:
import numpy as np

# Create the 'cover' array with the same shape as the dataframe
cover = np.zeros((len(sites), len(districts)), dtype=int)

# Iterate through the dataframe to fill the 'cover' array
for i in range(len(sites)):
    for j in range(len(districts)):
        cover[i][j] = df.iloc[i, j]  # Get the value from the dataframe and assign it to the cover array

# Print the 'cover' array to verify
print("cover array:")
print(cover)

In [58]:
!pip install -q pyomo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
!apt-get install -y -qq coinor-cbc

In [60]:
import pyomo.environ as pyo

In [62]:
model = pyo.ConcreteModel()

In [ ]:
model.x = pyo.Var(sites, within=pyo.Binary)
model.x.pprint()

In [ ]:
model.obj = pyo.Objective(
    expr = sum(model.x[i] for i in sites),
    sense = pyo.minimize )

model.obj.pprint()

In [ ]:
model.constraint_coverage = pyo.ConstraintList()
for j in districts:
  model.constraint_coverage.add(sum(model.x[i] for i in sites if cover[i][j]>0) >=1)
model.constraint_coverage.pprint()

In [ ]:
opt = pyo.SolverFactory('cbc')
model.pprint()

In [68]:
opt_solution = opt.solve(model)

In [ ]:
# Print the values of the decision variables
print("\nEMS sites opening:")
for i in sites:
    print(f"{i}: {model.x[i].value}")

print(f"Total number of sites: {model.obj()}")